In [25]:
import os
import time
import copy

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets

from livelossplot import PlotLosses

# Using model code from Stutz et.al.
os.chdir("/data/adv_geom/drg/")
from models.classifier import Classifier

In [2]:
TIME_STAMP = time.strftime("%d%m%Y-%H%M%S")

In [3]:
# Preparing ResNet50 for fine-tuning on MNIST
mnist_resnet = 


        
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

optimizer = optim.Adam(mnist_resnet.parameters(), lr=1e-6)


In [4]:
# Load the MNIST dataset
DATA_DIR = "/data/adv_geom/datasets"
BATCH_SIZE = 16

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(DATA_DIR, train=True, download=False, transform=transforms.Compose([
            transforms.ToTensor(),
            ])),
        batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(DATA_DIR, train=False, download=False, transform=transforms.Compose([
            transforms.ToTensor(),
            ])),
        batch_size=BATCH_SIZE, shuffle=True)

train_subset, val_subset = torch.utils.data.random_split(
        train_loader.dataset, [50000, 10000], generator=torch.Generator().manual_seed(1))

X_train = train_subset.dataset.data[train_subset.indices]
y_train = train_subset.dataset.targets[train_subset.indices]

X_val = val_subset.dataset.data[val_subset.indices]
y_val = val_subset.dataset.targets[val_subset.indices]

train_loader = torch.utils.data.DataLoader(dataset=train_subset, shuffle=True, batch_size=BATCH_SIZE)
val_loader = torch.utils.data.DataLoader(dataset=train_subset, shuffle=False, batch_size=BATCH_SIZE)


In [5]:
dataloaders = {"train": train_loader, "val": val_loader, "test": test_loader}
NAME = "mnist_ft"
DUMP_DIR = "/data/adv_geom/dumps/" + NAME + "_" + TIME_STAMP

if not os.path.exists(DUMP_DIR):
    os.mkdir(DUMP_DIR)

liveloss = PlotLosses(fig_path=DUMP_DIR + "/" + NAME)

In [7]:
criterion = nn.CrossEntropyLoss()

In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        
        logs = {}
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            #if phase == 'train':
            #    scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            prefix = ""
            if phase == "val":
                prefix = "val_"
                
            
            logs[prefix + "acc"] = epoch_acc
            logs[prefix + "loss"] = epoch_loss
            

            #print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            #    phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save({'epoch': epoch,'model_state_dict': model.state_dict(),\
                            'optimizer_state_dict': optimizer.state_dict()},\
                           DUMP_DIR + "/" + NAME + "_"+ TIME_STAMP + "_" + "_".join([str(logs["acc"]), str(logs["val_acc"])]) + ".pth")
        
        
        liveloss.update(logs)
        liveloss.draw()

#         print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [10]:
train_model(mnist_resnet, criterion, optimizer, None, num_epochs=100)

RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[16, 1, 28, 28] to have 3 channels, but got 1 channels instead

In [11]:
mnist_resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

ModuleNotFoundError: No module named 'common'